JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import joblib
import requests

API_BASE = "https://api.balldontlie.io/v1"

def fetch_all_stats(seasons=[2022, 2023], per_page=100):
    """Fetch all stats from the API, handling pagination automatically."""
    all_data = []
    page = 1

    while True:
        params = {
            "seasons[]": seasons,
            "per_page": per_page,
            "page": page,
        }
        url = f"{API_BASE}/stats"
        try:
            response = requests.get(url, params=params, timeout=10)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            raise SystemExit(f"❌ API request failed: {e}")

        try:
            data = response.json()
        except ValueError:
            raise SystemExit(f"❌ Could not decode JSON. Response:\n{response.text[:300]}")

        if "data" not in data or not isinstance(data["data"], list):
            raise SystemExit(f"❌ Unexpected API format: {data}")

        if not data["data"]:
            break  # No more pages

        all_data.extend(data["data"])
        page += 1

    return pd.json_normalize(all_data)

def main():
    print("📡 Fetching NBA stats...")
    games = fetch_all_stats(seasons=[2022, 2023])

    if games.empty:
        raise SystemExit("❌ No data returned from API. Try adjusting filters.")

    df = games[["pts", "ast", "reb"]].dropna()

    if df.empty:
        raise SystemExit("❌ No usable rows after dropping missing values.")

    print("✅ Sample training data:")
    print(df.head())

    # Train model
    X = df[["ast", "reb"]]
    y = df["pts"]
    model = LinearRegression()
    model.fit(X, y)

    joblib.dump(model, "nba_model.pkl")
    print("✅ Model trained and saved as nba_model.pkl")

if __name__ == "__main__":
    main()


📡 Fetching NBA stats...


SystemExit: ❌ API request failed: 401 Client Error: Unauthorized for url: https://api.balldontlie.io/v1/stats?seasons%5B%5D=2022&seasons%5B%5D=2023&per_page=100&page=1